<a href="https://colab.research.google.com/github/O-Kpy/Kaggle/blob/main/spooky_author_identification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install transformers

     |████████████████████████████████| 2.5 MB 8.2 MB/s 
     |████████████████████████████████| 895 kB 52.9 MB/s 
     |████████████████████████████████| 3.3 MB 82.0 MB/s 


In [19]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, SequentialSampler, random_split, TensorDataset, RandomSampler
from torch.optim import lr_scheduler

from transformers import BertConfig, BertModel, BertTokenizer, AdamW, DistilBertModel
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder

In [3]:
train = pd.read_csv('/content/drive/MyDrive/dataset/kaggle/nlp/spooky-author-identification/spooky-author-identification-train.csv', index_col='id')
test = pd.read_csv('/content/drive/MyDrive/dataset/kaggle/nlp/spooky-author-identification/spooky-author-identification-test.csv', index_col='id')
submit = pd.read_csv('/content/drive/MyDrive/dataset/kaggle/nlp/spooky-author-identification/spooky-author-identification-sample_submission.csv')
train.shape, test.shape

((19579, 2), (8392, 1))

In [4]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
train['author'] = LabelEncoder().fit_transform(train['author'])

In [5]:
train = train.reset_index(drop=True)
test = test.reset_index(drop=True)

In [6]:
x_train = train['text'].values
y_train = train['author'].values
x_test = test.copy()

In [8]:
# 토큰화와(Tokenizer) 마스크(mask)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

sequence_len = 64

x_train_tokens = []
train_attention_mask = []

for text in x_train:
  encoded_dict = tokenizer.encode_plus(text, 
                                       add_special_tokens=True,
                                       max_length=sequence_len, 
                                       padding='max_length',
                                       return_tensors='pt',
                                       truncation=True)
  x_train_tokens.append(encoded_dict['input_ids'])
  train_attention_mask.append(encoded_dict['attention_mask'])

x_train_tokens = torch.cat(x_train_tokens, dim=0)
train_attention_mask = torch.cat(train_attention_mask, dim=0)
y_train = torch.tensor(y_train)

In [9]:
x_train_tokens.size(), train_attention_mask.size(), y_train.size()

(torch.Size([19579, 64]), torch.Size([19579, 64]), torch.Size([19579]))

In [10]:
# Dataset, DataLoader
batch_size=16
dataset = TensorDataset(x_train_tokens.to(device),
                        y_train.long().to(device),
                        train_attention_mask.to(device)
                        )

train_size = int(0.8*len(dataset))
val_size = len(dataset) - train_size

train_set, valid_set = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_set,
                          sampler=RandomSampler(train_set),
                          batch_size=batch_size)
valid_loader = DataLoader(valid_set,
                          sampler=RandomSampler(valid_set),
                          batch_size=batch_size)

In [11]:
example_batch = next(iter(train_loader))

print('what is drawn from our dataloader?', type(example_batch))
print('\nfirst entry:', example_batch[0].size(), type(example_batch[0]), example_batch[0].dtype)
print("\nsecond entry: ", example_batch[1].size(), type(example_batch[1]), example_batch[1].dtype)

what is drawn from our dataloader? <class 'list'>

first entry: torch.Size([16, 64]) <class 'torch.Tensor'> torch.int64

second entry:  torch.Size([16]) <class 'torch.Tensor'> torch.int64


In [12]:
bert = BertModel.from_pretrained('bert-base-uncased')
bert.to(device)

batch_features = example_batch[0].to(device)
bert_output = bert(input_ids=batch_features)
print('bert output:', type(bert_output), len(bert_output))
print('first entry:', type(bert_output[0]), bert_output[0].size())
print('second entry:', type(bert_output[1]), bert_output[1].size())

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


bert output: <class 'transformers.modeling_outputs.BaseModelOutputWithPoolingAndCrossAttentions'> 2
first entry: <class 'torch.Tensor'> torch.Size([16, 64, 768])
second entry: <class 'torch.Tensor'> torch.Size([16, 768])


# https://www.kaggle.com/odedgolden/spooky-authors-analysis 참고해서 모델 완성 시켜보기

In [20]:
class DistilBertAuthorClassifier(nn.Module):
    def __init__(self):
        super(DistilBertAuthorClassifier, self).__init__()
        self.num_labels = 3

        self.softmax = nn.Softmax(dim=1)
        self.distilbert = DistilBertModel.from_pretrained('distilbert-base-cased')
        # self.pre_classifier = nn.Linear(config.hidden_size, config.hidden_size)
        self.classifier = nn.Linear(self.distilbert.config.dim, 3)
        self.dropout = nn.Dropout(0.3)

        nn.init.xavier_normal_(self.classifier.weight)

    def forward(self, input_ids, attention_mask):
        distilbert_output = self.distilbert(input_ids=input_ids,
                                            attention_mask=attention_mask)
        hidden_state = distilbert_output[0]
        # print(f'hidden_state shape: {hidden_state.shape}')                
        # print(f'hidden_state shape[2]: {hidden_state.shape[2]}')                
        pooled_output = hidden_state[:, 0, :]                   
        # pooled_output = self.pre_classifier(pooled_output)   
        # pooled_output = nn.ReLU()(pooled_output)             
        pooled_output = self.dropout(pooled_output)        
        logits = self.classifier(pooled_output)
        # logits = self.softmax(logits)
        return logits

In [13]:
class BertClassifier(nn.Module):
  def __init__(self, linear_size):
    super(BertClassifier, self).__init__()
    self.bert = BertModel.from_pretrained('bert-base-uncased')
    self.dropout1 = nn.Dropout()
    self.linear1 = nn.Linear(in_features=768, out_features=linear_size)
    self.relu1 = nn.ReLU()
    self.dropout2 = nn.Dropout(p=0.5)
    self.batch_norm1 = nn.BatchNorm1d(num_features=linear_size)
    self.linear2 = nn.Linear(in_features=linear_size, out_features=3)
    self.relu2 = nn.ReLU()
    self.batch_norm2 = nn.BatchNorm1d(num_features=3)
    self.sigmoid = nn.Sigmoid()

  def forward(self, tokens, attention_mask):
    bert_output = self.bert(input_ids=tokens, attention_mask=attention_mask)
    x = self.dropout1(bert_output[1])
    x = self.linear1(x)
    x = self.relu1(x)
    x = self.dropout2(x)
    x = self.batch_norm1(x)
    x = self.linear2(x)
    x = self.relu2(x)
    x = self.batch_norm2(x)
    proba = self.sigmoid(x)
    return proba

  def freeze_bert(self):
    for param in self.bert.named_parameters():
      param[1].requires_grad=False

  def unfreeze_bert(self):
    for param in self.bert.named_parameters():
      param[1].requires_grad=True

In [21]:
def eval(probas, y):
    preds_batch_np = probas.cpu().detach().numpy()
    y_np = y.cpu().detach().numpy()
    acc = accuracy_score(y_true=y_np, y_pred=preds_batch_np)
    return acc

def train(model, epochs):
  optimizer = AdamW(model.parameters(), lr=2e-5, eps=1e-8)
  loss_func = nn.CrossEntropyLoss()
  scheduler = lr_scheduler.ReduceLROnPlateau(optimizer=optimizer)
  history = []
  best_acc = 0

  for epoch in range(1, epochs+1):
    print("=== Epoch: ", epoch, " / ", epochs, " ===")
    print("LR:", optimizer.state_dict()["param_groups"][0]["lr"])

    # ---- train ----
    model.train()
    for batch in train_loader:
      x, y, mask = batch
      optimizer.zero_grad()
      probas = model(x, mask)
      loss = loss_func(probas, y)
      loss.backward()
      optimizer.step()

      running_loss = loss.item()
      print(f'epochs {epoch}, loss:{running_loss}')

    # ---- valid ----
    model.eval()
    acc_val_total = 0
    accumulated_val_loss = 0

    for batch in valid_loader:
      x, y, mask = batch
      with torch.no_grad():
        probas  = model(x, mask)
      loss = loss_func(probas, y)
      accumulated_val_loss += loss
      acc = eval(probas, y)
      acc_val_total += acc
    avg_val_loss = accumulated_val_loss / len(valid_loader)
    scheduler.step(avg_val_loss)
    acc_val_total = acc_val_total / len(valid_loader)

    print("Val Acc:", acc_val_total)
    print(f'Val_loss:{avg_val_loss}')

    if acc_val_total > best_acc:
      best_acc = acc_val_total
      torch.save(model, 'best.pt')
      print('model saved...')

  return torch.load("best.pt")

In [22]:
epochs=30
freeze_epochs = [2,4,6,8,10,12,14,16,18,20,22,24,26,28]
unfreeze_epochs = [3,5,7,9,11,13,15,17,19,21,23,25,27,29]

bert_clf = DistilBertAuthorClassifier().to(device)

bert_clf = train(model=bert_clf,
                 epochs=epochs)

Some weights of the model checkpoint at distilbert-base-cased were not used when initializing DistilBertModel: ['vocab_transform.bias', 'vocab_projector.weight', 'vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_transform.weight', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


=== Epoch:  1  /  30  ===
LR: 2e-05


RuntimeError: ignored